In [1]:
import os
import logging
import threading
import pandas as pd  
import math
import dill
import numpy as np  
import matplotlib.pyplot as plt
import pydot
import seaborn as seabornInstance
import networkx
from sqlalchemy import Column, Integer, String, Float, DateTime, Boolean, func
from iotfunctions import base
from iotfunctions import anomaly
from iotfunctions import bif
from iotfunctions import entity
from iotfunctions import metadata
from iotfunctions.metadata import EntityType
from iotfunctions.db import Database
from iotfunctions.enginelog import EngineLogging
from iotfunctions import estimator, ui, base, bif
from iotfunctions.base import BaseTransformer
from iotfunctions.ui import (UISingle, UIMultiItem, UIFunctionOutSingle,
                 UISingleItem, UIFunctionOutMulti, UIMulti, UIExpression,
                 UIText, UIStatusFlag, UIParameters)
from iotfunctions.enginelog import EngineLogging
from iotfunctions import pipeline as pp
from iotfunctions.stages import DataWriterSqlAlchemy, DataWriterFile
from iotfunctions.pipeline import JobController, DataAggregator

import datetime as dt
from scipy.linalg import norm
import scipy as sp
from scipy import signal
from scipy import linalg
from pyod.models.cblof import CBLOF
from pyod.models.ocsvm import OCSVM
from pyod.models.xgbod import XGBOD

#import pmdarima as pm
import inverse_covariance as icov # from package skggm
from inverse_covariance import QuicGraphLasso

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

from skimage import util as skiutil # for nifty windowing

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import LSTM, Dense, Flatten, Input, Dropout, Convolution1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from keras.utils import model_to_dot
import telemanom
from telemanom.helpers import Config
from telemanom.errors import Errors
import telemanom.helpers as helpers
from telemanom.channel import Channel
from telemanom.modeling import Model

%matplotlib inline

/home/markus/.local/lib/python3.7/site-packages/iotfunctions/bif.py:1605: UserWarning: IoTCalcSettings is deprecated. Use entity type constants instead of a metadata provider to set entity type properties
  warnings.warn(('IoTCalcSettings is deprecated. Use entity type constants'
/home/markus/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/markus/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised wh

In [2]:
conf = Config("/home/markus/src/mmfunctions/telemanom/config.yaml")

In [3]:
chan = Channel(conf, "T-9")

In [4]:
chan.load_data("./telemanom")
chan.train

array([[-0.33332882,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.33332882,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.33332882,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.33333784,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33333784,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.33333784,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [5]:
# split training data into windows of size 260: 
#   lookback aka l_s (defined in config.yaml as 250) + n_predictions (defined in config.yaml as 10)
#   to predict the last 10 data points from the past 250 ones
chan.shape_data(chan.train, train=True)

In [6]:
print(chan)

Here!

Channel:Channel
Training data shape: (179, 250, 55), (179, 10)
  Test data shape: (836, 250, 55), (836, 10)
  Original data shape: (439, 55), (1096, 55)


In [7]:
model = Model(conf, "T-9", chan, "./telemanom")

Train on 143 samples, validate on 36 samples
Epoch 1/35
143/143 [==============================] - 1s 9ms/step - loss: 0.1141 - val_loss: 0.0763
Epoch 2/35
143/143 [==============================] - 1s 5ms/step - loss: 0.0636 - val_loss: 0.0189
Epoch 3/35
143/143 [==============================] - 1s 4ms/step - loss: 0.0286 - val_loss: 0.0080
Epoch 4/35
143/143 [==============================] - 1s 5ms/step - loss: 0.0214 - val_loss: 0.0132
Epoch 5/35
143/143 [==============================] - 1s 4ms/step - loss: 0.0199 - val_loss: 0.0166
Epoch 6/35
143/143 [==============================] - 1s 4ms/step - loss: 0.0199 - val_loss: 0.0108
Epoch 7/35
143/143 [==============================] - 1s 5ms/step - loss: 0.0157 - val_loss: 0.0078
Epoch 8/35
143/143 [==============================] - 1s 5ms/step - loss: 0.0155 - val_loss: 0.0072
Epoch 9/35
143/143 [==============================] - 1s 5ms/step - loss: 0.0139 - val_loss: 0.0088
Epoch 10/35
143/143 [==============================] - 

In [11]:
print (model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 80)          43520     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 80)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 80)                51520     
_________________________________________________________________
dropout_2 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 95,850
Trainable params: 95,850
Non-trainable params: 0
__________________________________________________

In [12]:
df = pd.DataFrame(chan.train)
df.describe()
# P-10,MSL,"[[4590, 4720]]",[point],6100
#df.iloc[2750:2800]

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
count,439.000000,439.0,439.0,439.0,439.0,439.000000,439.000000,439.0,439.0,439.0,...,439.0,439.0,439.0,439.0,439.0,439.0,439.0,439.0,439.0,439.0
mean,0.272594,0.0,0.0,0.0,0.0,0.006834,0.004556,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.202359,0.0,0.0,0.0,0.0,0.082477,0.067420,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,-1.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.333338,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.333338,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.333338,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
